# NYC Buildings

Many plotting libraries can handle collections of polygons, e.g. [Bokeh](https://docs.bokeh.org/en/latest/docs/examples/topics/geo/texas_hover_map.html) or [HoloViews+Bokeh](https://holoviews.org/gallery/demos/bokeh/choropleth_data_link.html). However, because browser-based libraries like Bokeh and Plotly send all the polygon data to the browser, they can struggle when either the collections or the polygons themselves get large. Even natively in Python, typical formats like Shapely for representing polygons scale poorly to large polygon collections, because each polygon is wrapped up as a separate Python object, leading to a lot of duplicated storage overhead when many polygons of the same type are defined.

If you want to work with lots of polygons, here you can see how to use [SpatialPandas](https://github.com/holoviz/spatialpandas) and Dask to represent polygons efficiently in memory, fastparquet to represent them efficiently on disk, and [Datashader](https://datashader.org) to render them quickly in a web browser. This notebook also demonstrates how to support hovering for datashaded polygons, with Bokeh overlaying a single vector-based representation of a polygon where the mouse cursor is, while all the rest are sent to the browser only as rendered pixels. That way hover and other interactive features can be supported fully without ever needing to transfer large amounts of data or store them in the limited memory of the web browser tab.  

This example plots the outlines of all the buildings in New York City. See
[nyc.gov](https://nyc.maps.arcgis.com/home/item.html?id=870bf69e8a8044aea4488e564c0b4010#overview) for the original data and its description.

In [ ]:
import hvplot.dask # noqa
import hvplot.pandas # noqa
import geopandas as gpd
import colorcet as cc

In [ ]:
gdf = gpd.read_parquet('new_nyc_buildings.parq')

gdf.head()

Now we compute the top categories and drop everything else:

In [ ]:
cats = list(gdf['type'].value_counts().iloc[:10].index) + ['unknown']
gdf['type'] = gdf['type'].fillna('unknown')
gdf = gdf[gdf['type'].isin(cats)]
gdf['type'] = gdf['type'].astype('category')

Next we build a legend for the categories and declare a tile source as backdrop:

In [ ]:
colors    = cc.glasbey_bw_minc_20_maxl_70
color_key = {cat: tuple(int(e*255.) for e in colors[i]) for i, cat in enumerate(cats)}

Now we put it all together, declaring a `Polygons` element from our data, datashade them and use the `inspect_polygons` operation to allow us to hover on the data:

In [ ]:
gdf.head(1000).hvplot.polygons(tiles='CartoLight', rasterize=True)

In [ ]:
gdf.hvplot.polygons(tiles='CartoLight', rasterize=True)

In [ ]:
# This is the hold-up point
# gdf.head(1000).hvplot.polygons(tiles='CartoLight', rasterize=True, c='type', cmap=color_key)

Finally we will plot each category of buildings separately: